In [1]:
# BASE CASE: load the viewer with desired options
from vedo import *

# DEBUG MODE, not using pip install here
import os
import sys
sys.path.append(os.path.abspath('../'))
from iblviewer import *

resolution = 50 # units = um
controller = atlas_controller.AtlasController()
controller.initialize(resolution, embed_ui=True, jupyter=True)
#controller.render()

INFO:root:Starting IBL Viewer
INFO:root:[AtlasModel] origin: [5739 5400  332]
INFO:root:[AtlasModel] resolution: 50
INFO:root:Loaded atlas volume with Allen mapping
INFO:root:

Starting brain atlas View...

INFO:root:Render info: VTK is not built with OSPRay support. Using OpenGL.


In [2]:
# DEMO 0: interact with the viewer once it's started. 
# Currently only works in Visual Studio with a minor caveat and slower UI.
controller.toggle_dorsal_view()

In [32]:
# DEMO 1: add insertion probes data from IBL database (DataJoints)
from oneibl.one import ONE
import numpy as np
from vedo import *
from iblviewer import *

import numpy as np
import random
import vedo


def get_bwm_ins_alyx(one):
    """
    Return insertions that match criteria :
    - project code
    - session QC not critical (TODO may need to add probe insertion QC)
    - at least 1 alignment
    - behavior pass
    :param one: "one" connection handler
    :return:
    ins: dict containing the full details on insertion as per the alyx rest query
    ins_id: list of insertions eids
    sess_id: list of (unique) sessions eids
    """
    ins = one.alyx.rest('insertions', 'list',
                        provenance='Ephys aligned histology track',
                        django='session__project__name__icontains,ibl_neuropixel_brainwide_01,'
                               'session__qc__lt,50,'
                               'json__extended_qc__alignment_count__gt,0,'
                               'session__extended_qc__behavior,1')
    
    ins_id = [item['id'] for item in ins]
    sess_id = [item['session_info']['id'] for item in ins]
    # Here's what's in 'json':
    # dict_keys(['qc', 'n_units', 'xyz_picks', 'extended_qc', 'drift_rms_um', 'firing_rate_max', 'n_units_qc_pass', 
    # 'amplitude_max_uV', 'firing_rate_median', 'amplitude_median_uV', 'whitening_matrix_conditioning'])
    xyz_picks = {}
    for item in ins:
        ins_id = item['id']
        picks = np.array(item['json'].get('xyz_picks', []))
        xyz_picks[ins_id] = picks
    sess_id = np.unique(sess_id)
    return xyz_picks


def get_picks_mean_vectors(xyz_picks, extent=3):
    """
    Get a mean vector from picks coordinates
    :param xyz_picks: Dictionary xyz picks, the key being the identifier for that data set
    :param extent: Number of points to take from start and end for mean computation of end points
    :return: 3D numpy array and a list of ids
    """
    vectors = []
    ids = []
    # Mean between first and last three picks
    for ins_id in xyz_picks:
        raw_picks = xyz_picks[ins_id]
        end_pt = np.mean(raw_picks[-extent:], axis=0)
        start_pt = np.mean(raw_picks[:extent], axis=0)
        vectors.append([start_pt, end_pt])
        ids.append(ins_id)
    return np.array(vectors), ids


def add_insertion_probes(controller, one, reduced=True, with_labels=True):
    """
    Add insertion probe vectors
    :param controller: The IBLViewer controller
    :param one: The "one" connection to IBL server
    :param reduced: Whether insertion probes should be reduced to simple lines
    :param with_labels: Whether labels should be added to the lines
    """
    vectors = get_bwm_ins_alyx(one)
    if reduced:
        vectors, ids = get_picks_mean_vectors(vectors)
        lines = controller.view.new_segments(vectors)
    else:
        lines = controller.view.new_lines(vectors)
    actors = [lines]
    
    if with_labels:
        labels = lines.labels('ids', rotX=-180, cells=True)
        actors.append(labels)

    controller.plot.add(actors)
    return lines


one = ONE(base_url="https://alyx.internationalbrainlab.org")
add_insertion_probes(controller, one)

f70-9a0f-65b3e3cd6d4a': [array([-2563.        , -1683.33333333,  -942.33333333]), array([-2071.33333333, -2049.        , -3534.        ])], '6d3b68e0-3efd-4b03-b747-16e44118a0a9': [array([  735.66666667, -1883.        ,   -18.33333333]), array([-1355.33333333, -2633.33333333, -5792.66666667])], '701026df-e170-4ca7-88aa-eb0b95ef6ba1': [array([-2929.66666667, -3174.66666667,  -767.66666667]), array([-3179.66666667, -3966.33333333, -5767.66666667])], '71a92c54-69f0-488b-ae2a-cb6c1524233c': [array([-3604.66666667, -3725.        , -1075.66666667]), array([-4397.        , -4091.33333333, -5134.33333333])], '74006cce-e266-4ebb-aaab-3fd5eb2d72e5': [array([-1330.        , -2549.        ,   -18.33333333]), array([-2429.66666667, -3032.33333333, -5768.        ])], '749cb2b7-e57e-4453-a794-f6230e4d0226': [array([-1505.        ,  2399.33333333, -1459.33333333]), array([ -546.66666667,  1765.66666667, -4934.        ])], '76de0e1a-30aa-4713-9fe5-25ad2dff653f': [array([-1288.33333333, -2066.        , 

In [1]:
# DEMO 2: connect to IBL database and fetch 
# connectivity data from Leenoy and Alessandro's work


# Invoke your own code with the lines below but you should wrap it in
# properly encapsulated code...
%%bash
%git clone https://github.com/int-brain-lab/friendly_neurons.git
%load friendly_neurons/data_analysis.py

print(general_analysis)

def add_point_neurons(controller, data, with_labels=False, verbose=False):
    regions_data = list()
    positions = list()
    timings = list()
    # data structure x | y | z | region | pre-time allegiance | during-time allegiance | post-time allegiance 
    # but data is dirty, with a big list of nested np.arrays with Decimal type values, then strings, and ints
    for point_id in range(len(data)):
        entry = data[point_id]
        if isinstance(entry[0], int):
            x = float(entry[0])
            y = float(entry[1])
            z = float(entry[2])
        else:
            # For some reason, initial data gives me 3 values for x, 3 for y, 3 for z 
            # (Alessandro says to use the first ones)
            x = -float(entry[0][0])
            y = -float(entry[1][0])
            z = -float(entry[2][0])
        positions.append([x, y, z])
        regions_data.append(entry[3])
        timings.append([entry[-3], entry[-2], entry[-1]])
        
    regions_data = np.array(regions_data)
    positions = np.array(positions).astype(np.float64)
    timings = np.array(timings)
    if verbose:
        print('Pre range', min(timings[:, 0]), max(timings[:, 0]))
        print('During range', min(timings[:, 1]), max(timings[:, 1]))
        print('Post range', min(timings[:, 2]), max(timings[:, 2]))
    
    #three_colors = ['red', 'blue', 'yellow'] #['#ffbdbd', '#c9c9ff', '#e1f7d5']
    adj_positions = positions[:] * [1, 1, 1]
    adj_positions += np.random.rand(len(adj_positions), 3) * 100
    values = timings[:, 0]

    # We approximate the neuron soma size to 5um here
    #point_neurons = Points(points, r=5).lighting(0).pickable(True).cmap('Accent', values, on='cells')
    #point_neurons.GetProperty().RenderPointsAsSpheresOn()
    points = controller.view.add_points(adj_positions, values)
    actors = [points]
    if with_labels:
        # Using functions from vedo (easy-to-use wrapper on top of VTK)
        labels = points.labels('id', cells=True)
        actors.append(labels)
    controller.plot.add(actors)
    return points


#add_point_neurons(controller)

UsageError: Line magic function `%%bash` not found.


In [2]:
 # DEMO 3: load custom pickle data with scalar values per brain region 
# (some regions have no corresponding value) and fake a time series with this

import vedo
import random
import pickle
import pandas as pd

# Data given by Berk
def process_priors(controller, file_path='./data/completefits_2020-11-09.p', randomize=None):
    """
    Process priors data and get color map and scalar values
    """
    pickles = []
    with (open(file_path, 'rb')) as openfile:
        while True:
            try:
                pickles.append(pickle.load(openfile))
            except EOFError:
                break
    df = pickles[0]
    filtered_df = df['rawpoints'].groupby('acronym').agg({'value':'median'})
    
    min_value = float(np.amin(filtered_df, axis=0).to_numpy()[0])
    max_value = float(np.amax(filtered_df, axis=0).to_numpy()[0])
    print('Min prior value ' + str(min_value))
    print('Max prior value ' + str(max_value))

    scalars_map = {}
    for acronym, value in filtered_df.iterrows():
        region_id, row_id = controller.model.get_region_and_row_id(acronym)
        if row_id == 0:
            # We ignore void acronym
            continue
        scalars_map[int(row_id)] = float(value.to_numpy()[0])

    return scalars_map


def get_color_map(controller, scalar_map, nan_alpha=0.0, seed=None):
    """
    Get a color map
    """
    if seed is not None:
        random.seed(seed)
    rgb = []
    alpha = []

    # Init all to clear gray (90% white)
    #c = np.ones((self.metadata.id.size, 4)).astype(np.float32) * 0.9
    #c[:, -1] = 0.0 if only_custom_data else alpha_factor
    #print('Assigning', values.size, 'to atlas ids', self.metadata.id.size)

    for r_id in range(len(controller.model.metadata)):
        rgb.append([r_id, [1.0, 1.0, 1.0]])
        a = nan_alpha if r_id > 0 else 0.0
        alpha.append([r_id, a])

    values = scalar_map.values()
    min_p = min(values)
    max_p = max(values)
    rng_p = max_p - min_p
    
    for row_id in scalar_map:
        value = scalar_map[row_id]
        '''
        hsv_col = matplotlib.colors.rgb_to_hsv(color)
        hsv_col[1] = 0.0
        desaturated_color = matplotlib.colors.hsv_to_rgb(hsv_col)
        '''
        if seed is not None and seed > 0:
            value = value + random.random() * rng_p
        #else:
            #value = float(value.to_numpy()[0])
        rgb[row_id] = [row_id, list(vedo.colorMap(value, 'viridis', min_p, max_p))]
        alpha[row_id] = [row_id, 1.0]

    return rgb, alpha


def load_priors_in_viewer(controller, nan_alpha=0.0, fake_time_series_steps=0):
    """
    Load priors into the viewer, faking a time series from there
    """
    scalar_map = process_priors(controller)
    for index in range(fake_time_series_steps + 1):
        rgb, alpha = get_color_map(controller, scalar_map, nan_alpha, index)
        controller.add_transfer_function(scalar_map, rgb, alpha, make_current=False)


load_priors_in_viewer(controller, nan_alpha=0.0, fake_time_series_steps=100)
controller.render()

Min prior value -0.002318870876007073
Max prior value 0.027804735660474256


In [3]:
# DEMO 4: export a movie sequence
# TODO

INFO:root:
Visualizer started: 42.878658s




In [ ]:
























# VEDO test-case

from vedo import *
embedWindow('panel') # or k3d

s1 = Sphere().flat() # flat shading
s2 = Cube(pos=(1,0,0), c='white', alpha=0.2)

# add scalars to the sphere that correspond to their distance from the cube
s1.distanceToMesh(s2, signed=True, negate=False)

#s1.printInfo()
#print(s1.getPointArray("Distance"))

show(s1, s2, viewup='z')